#### Импорты

In [1]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import pandas as pd

#### Краулер

Для создания датасета я решила собрать отзывы о различных тональных средствах с сайта irecommend.ru

In [2]:
# чтобы обойти блокировку, притворимся нормальным браузером
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}
print(headers)

{'User-Agent': 'Mozilla/5.0 (X11; U; Linux i686 (x86_64); en-US; rv:1.8.0.5) Gecko/20060726 Red Hat/1.5.0.5-0.el4.1 Firefox/1.5.0.5 pango-text'}


Напишем функцию errors_handler, обрабатывающую ошибки при парсинге страниц с отзывами.

In [3]:
def errors_handler(page_number, review):

    with open('./errors.txt', 'a', encoding='utf-8') as f:
        
        f.write(f'Ошибка возникла на странице {page_number}\n')
        f.write(f'Текст ревью:\n {review}\n\n')

Функция get_review необходима для парсинга собственно самой страницы с отзывом.

In [7]:
def get_review(review_link, page_number):
    
    global reviews_dict

    link = review_link.attrs['href']
    url = 'https://irecommend.ru' + link
    session_review = requests.session()
    req = session_review.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')

    # текст отзыва
    review = soup.find('div', {'itemprop': 'reviewBody'})
    try:
        text = review.text
        reviews_dict['Отзыв'].append(text)
    except AttributeError:
        reviews_dict['Отзыв'].append('')
        errors_handler(page_number, review)

    # оценка 
    rating = soup.find('meta', {'itemprop': 'ratingValue'})
    try:
        grade = rating.attrs['content']
        reviews_dict['Оценка'].append(grade)
    except AttributeError:
        reviews_dict['Оценка'].append('')
        errors_handler(page_number, review)


В разделе "Тональные средства" на сайте 100 страниц с отзывами. Возьмем 15 из них, по каждой будем проходиться, собирая ссылки на страницы отдельных отзывов и передавая их в функцию get_review

In [8]:
def get_page(page_number):
    
    session_page = requests.session()
    response = session_page.get(f'https://irecommend.ru/catalog/reviews/30-884?page={page_number}', headers=headers)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    reviews_links = soup.find_all('a', {'class': 'more'})

    for review_link in reviews_links:
        
        get_review(review_link, page_number)

    print(f'Page {page_number} has been completed')

Запускаем проход по страницам в цикле:

In [9]:
reviews_dict = {'Отзыв' : [], 'Оценка' : []}
for i in range(30):
    get_page(i)

Page 0 has been completed
Page 1 has been completed
Page 2 has been completed
Page 3 has been completed
Page 4 has been completed
Page 5 has been completed
Page 6 has been completed
Page 7 has been completed
Page 8 has been completed
Page 9 has been completed
Page 10 has been completed
Page 11 has been completed
Page 12 has been completed
Page 13 has been completed
Page 14 has been completed
Page 15 has been completed
Page 16 has been completed
Page 17 has been completed
Page 18 has been completed
Page 19 has been completed
Page 20 has been completed
Page 21 has been completed
Page 22 has been completed
Page 23 has been completed
Page 24 has been completed
Page 25 has been completed
Page 26 has been completed
Page 27 has been completed
Page 28 has been completed
Page 29 has been completed


Полученные результаты сохраняем в формате csv-файла.

In [9]:
result = pd.DataFrame(reviews_dict)
result.to_csv('./result.csv')